# 🎓 How do I send an API request?

1. How can I reverse engineer 'the correct' API request when there is no documentation?

   - Monitor network traffic
   - examine the [Java CLI](https://domo-support.domo.com/s/article/360043437733?language=en_US)
   - see [Jae Wilsons TreasureTrove](./README.md#helpful-links)

2. How does the internet handle authentication?

   - pass authentication in cookies
   - pass authentication via request headers

3. **Domo uses a "username and password" flow to generate a session_token which is then used to authenticate most downstream API requests**
   - The session_token (like cookies) has an expiration date.


## ▶️ How to Monitor Network Traffic

1. Go to Data > Accounts to view a list of all accounts you have access to.

2. Monitor network traffic (different for each browser -- Inspect > Network on Chrome) to see what API request is being sent to retrieve a list of auth objects.

3. Find the API request.
   - Filter for "providers".
   - Examine the URL, headers, request Method (GET, PUT, POST, DELETE) and body<br>
   - Copy the query into a text editor.


### Auth examples with different curl requests


### ▶️ convert an API request into a function

Using functions improves legibility, recycle-ability, and maitainability of code

- Use the `requests` library to send an API request to the providers endpoint.
- modify the below function to receive `headers: dict` and modify the request.
- test by scraping your browser cookie and passing it into `headers`

`headers = {'cookie': <your_header>}`


In [1]:
DOMO_INSTANCE = (
    "domo-community"  ## it's common practice to declare constants using all caps.
)
COOKIE = "cookie_goes_here"

In [2]:
# %pip install requests

In [3]:
import requests


# fix me! solution at solutions/get_acounts_v1.py
def get_accounts(domo_instance):
    # v1 does not accept authentication and therefore will fail

    url = f"https://{domo_instance}.domo.com/api/data/v2/datasources/providers"

    return requests.request(method="GET", url=url)




In [4]:
# from  solutions.get_accounts_v1 import get_accounts

res = get_accounts(domo_instance=DOMO_INSTANCE)
res

<Response [401]>

### 🎓 What is <Response [401]>

1. requests.request [returns an istance of the Response class](https://www.geeksforgeeks.org/response-request-python-requests/)

2. classes are objects `dict` that have attributes (fields) that can be access with dot notation and methods (functions)
   - the Response class has attributes like `status_code` and method `.json()`

💡 DEVELOPER_TIP

- get comfortable googling "example of sending an API request with the requests library"
- use the type() function to see the data type of the response


In [5]:
# from the error message it's clear we need to pass authentication with our API request
print(type(res))


print({"status": res.status_code, "is_success": res.ok, "response": res.json()})

<class 'requests.models.Response'>
{'status': 401, 'is_success': False, 'response': {'status': 401, 'statusReason': 'Unauthorized', 'path': '/api/data/v2/datasources/providers', 'message': 'Full authentication is required to access this resource', 'toe': 'D2RFR14QKK-0FWIV-QTGGZ'}}


### 🧪 Extra Credit - uses classes as function oututs to guarantee code consistency

- create a `@dataclass ResponseClass` to use as the output of `get_accounts`
- with attributes `status: int, is_success: bool , response: dict`
- with a classmethod that converts a Response into a ResponseClass

This ensures that each function in your codebase always returns the same class even if you switch do a different library (like HTTPX).

It makes implementing error handling consistent because even if your transport library changes, you always have the same output class


In [6]:
# from solutions.ResponseClass import ResponseClass


# finish me
@dataclass
class ResponseClass:
    status: int
    # parameters go here.

    @classmethod
    def from_request_response(cls, res: requests.models.Response):
        pass


r = ResponseClass.from_request_response(res)
r

NameError: name 'dataclass' is not defined

## Use Full Auth (username and password) workflow to handle Domo API Authentication

"full authentication" is not the same as client_id and secret authentication which can be retrieved from <https://developer.domo.com>

- client_id and secret auth (same as developer_token authentication) only applies to ["public" APIs](https://www.youtube.com/watch?v=hRwrZABP8RE).

"full authentication" actually has two flavors. (see curl reques examples above)


### ▶️ generate a function to handle auth flow
- function should receive: `domo_instance, domo_username, domo_password, and return_raw: bool = False`

- get_full_auth should parse the response and return just the `sessionToken`

💡 DEVELOPER_TIP - You can avoid documentation IF your naming convention document your assumptions
    - document the type(s) we expect to receive for each input and output variable.
    - provide a document string (denoted in triple quotes) to describe what the function does

💡 implement tests to ensure expected / assumed behavior
    - can you test for 400 - 500 errors? (res.ok)
    - what if you get a 200 response but sent an invalid password? 

💡 to support debugging, add a test before parsing the response `if return_raw: return res`
    - this way if your API is ever acting strangely you can validate that the API is still returning the same object (i.e. they didn't relable sessionToken to session_token) 



In [ ]:
# fix me
def get_full_auth(
    return_raw: bool = False
) -> str:  # returns a session token
    """use exchanges username and password for a session_token"""

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password,
    }

    res = requests.request(method="POST", url=url, json=body, verify = False) 

    if return_raw:
        return res

    return res

In [ ]:
# from solutions.get_full_auth_v1 import get_full_auth

get_full_auth(
    domo_instance = 'domo-community',
    domo_username = 'dp24@test.com',
    domo_password = 'thisisinsecure',
)

TypeError: get_full_auth() got an unexpected keyword argument 'domo_instance'

### ▶️ modify get_accounts() to handle session_auth.

1. modify get_accounts to receive `headers: dict = None, session_token : str = None`

2. combine headers and session_token and pass to request

3. retrieve `session_token` from `get_full_auth()` then pass it to your modified `get_accounts`

### ▶️ Put it all together!

In [ ]:
# get_accounts goes here

In [7]:

# from solutions.get_accounts_v2 import get_accounts
# from solutions.get_full_auth_v1 import get_full_auth

session_token = get_full_auth(
    domo_instance = 'domo-community',
    domo_username = 'dp24@test.com',
    domo_password = 'thisisinsecure',
)

get_accounts(domo_instance=DOMO_INSTANCE, session_token=session_token)

d:\GitHub\domopalooza-24\.venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domo-community.domo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>